# Random Forest

## Setting Up

In [1]:
import pandas as pd 
import numpy as np

In [2]:
import sklearn
assert sklearn.__version__ >= "0.20"
import yfinance as yf

np.random.seed(42)

from sklearn.svm import LinearSVC 
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, f1_score, precision_score

from sklearn.model_selection import RandomizedSearchCV
from imblearn.over_sampling import RandomOverSampler

In [3]:
metrics_df = pd.DataFrame(columns =['Model', 'Accuracy', 'Precision', 'Recall', 'F1', 'Confusion Matrix'])

## Import Data 

In [4]:
X_train = pd.read_excel("../data/model_inputs/X_train.xlsx")
y_train = pd.read_excel("../data/model_inputs/y_train.xlsx")

X_test = pd.read_excel("../data/model_inputs/X_test.xlsx")
y_test = pd.read_excel("../data/model_inputs/y_test.xlsx")

In [5]:
X_train = X_train.rename(columns ={'Unnamed: 0': 'Date'})
X_train = X_train.set_index('Date')

y_train = y_train.rename(columns ={'Unnamed: 0': 'Date'})
y_train = y_train.set_index('Date')

X_test = X_test.rename(columns ={'Unnamed: 0': 'Date'})
X_test = X_test.set_index('Date')

y_test = y_test.rename(columns ={'Unnamed: 0': 'Date'})
y_test = y_test.set_index('Date')

### RANDOM OVERSAMPLING RESAMPLE

In [6]:
def random_oversampler(X_train, y_train):
    oversample = RandomOverSampler(sampling_strategy='not majority')
    X_over, y_over = oversample.fit_resample(X_train, y_train)
    return X_over, y_over

### Random Resample

In [7]:
data_resample = random_oversampler(X_train, y_train)

In [8]:
X_train = data_resample[0]
y_train = data_resample[1]

In [9]:
y_train['decision'].value_counts()

 0    1383
 1    1383
-1    1383
Name: decision, dtype: int64

## Initializing Random Forest

In [10]:
rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, random_state=42)

### Fitting Data

In [11]:
rnd_clf.fit(X_train, y_train)

C:\Users\chloe\AppData\Local\Temp\ipykernel_19964\1831517094.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rnd_clf.fit(X_train, y_train)


RandomForestClassifier(max_leaf_nodes=16, n_estimators=500, random_state=42)

### Predicting on train set

In [12]:
model = 'Random Forest TRAIN'
y_predict = rnd_clf.predict(X_train)
cm = confusion_matrix(y_train, y_predict)
accuracy = accuracy_score(y_train, y_predict)
precision = precision_score(y_train, y_predict, average = "weighted")
recall = recall_score(y_train, y_predict, average = "weighted")
f1 = f1_score(y_train, y_predict, average = "weighted")

In [13]:
int_metric = pd.DataFrame([[model, accuracy, precision, recall, f1, cm]], 
                          columns = ['Model', 'Accuracy', 'Precision', 'Recall', 'F1', 'Confusion Matrix'])

metrics_df = metrics_df.append(int_metric)

In [14]:
print("Confusion_Matrix:\n{}"
      .format(cm))
print("Accuracy_Score: {}\n"
      .format(accuracy))
print("Precision_Score: {}\n"
      .format(precision))
print("Recall Score: {}\n"
      .format(recall))
print("F1 Score: {}\n"
      .format(f1))

Confusion_Matrix:
[[1316   67    0]
 [  49 1266   68]
 [   0    0 1383]]
Accuracy_Score: 0.9556519643287539

Precision_Score: 0.9556585889655719

Recall Score: 0.9556519643287539

F1 Score: 0.9553488137439927



### Predicting on test set

In [15]:
model = 'Random Forest TEST'
y_predict = rnd_clf.predict(X_test)
cm = confusion_matrix(y_test, y_predict)
accuracy = accuracy_score(y_test, y_predict)
precision = precision_score(y_test, y_predict, average = "weighted")
recall = recall_score(y_test, y_predict, average = "weighted")
f1 = f1_score(y_test, y_predict, average = "weighted")

In [16]:
int_metric = pd.DataFrame([[model, accuracy, precision, recall, f1, cm]], 
                          columns = ['Model', 'Accuracy', 'Precision', 'Recall', 'F1', 'Confusion Matrix'])

metrics_df = metrics_df.append(int_metric)

In [17]:
print("Confusion_Matrix:\n{}"
      .format(cm))
print("Accuracy_Score: {}\n"
      .format(accuracy))
print("Precision_Score: {}\n"
      .format(precision))
print("Recall Score: {}\n"
      .format(recall))
print("F1 Score: {}\n"
      .format(f1))

Confusion_Matrix:
[[  6  15   0]
 [  5 131   1]
 [  1   9   0]]
Accuracy_Score: 0.8154761904761905

Precision_Score: 0.7517089093701997

Recall Score: 0.8154761904761905

F1 Score: 0.7771489355393465



## RandomizedSearchCV Tuning Random Forest
### Parameters includes: n_estimators, max_features, max_depth, min_samples_split, min_samples_leaf, bootstrap

In [38]:
n_estimators = [int(x) for x in np.linspace(start = 2, stop = 2000, num = 10)]
max_features = ['auto','sqrt']
max_depth = [int(x) for x in np.linspace(10,110, num = 11)]
min_samples_split = [2,5,10]
min_samples_leaf = [1,2,4]
bootstrap = [True, False]

params = {'n_estimators': n_estimators, 'max_features': max_features, 
          'max_depth': max_depth, 'min_samples_split': min_samples_split, 
         'min_samples_leaf':min_samples_leaf, 'bootstrap': bootstrap}
rs = RandomizedSearchCV(rnd_clf, params, random_state = 42,  n_iter = 200)
rs.fit(X_train, y_train)

C:\Python39\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Python39\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Python39\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Python39\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change

C:\Python39\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Python39\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Python39\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Python39\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change

C:\Python39\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Python39\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Python39\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Python39\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change

C:\Python39\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Python39\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Python39\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Python39\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has be

C:\Python39\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Python39\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Python39\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Python39\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has be

...

C:\Python39\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Python39\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Python39\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Python39\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-

C:\Python39\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Python39\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Python39\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Python39\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has be

C:\Python39\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Python39\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Python39\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Python39\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change

C:\Python39\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Python39\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Python39\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Python39\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has be

RandomizedSearchCV(estimator=RandomForestClassifier(max_leaf_nodes=16,
                                                    n_estimators=500,
                                                    random_state=42),
                   n_iter=200,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [2, 224, 446, 668, 890,
                                                         1112, 1334, 1556, 1778,
                                                         2000]},
                   random_state=42)

In [39]:
rs.best_params_

{'n_estimators': 890,
 'min_samples_split': 5,
 'min_samples_leaf': 4,
 'max_features': 'sqrt',
 'max_depth': 90,
 'bootstrap': False}

In [40]:
rs.best_score_

0.8958562355574287

## Initialising tuned Random Forest

In [18]:
rnd_clf = RandomForestClassifier(n_estimators=890, min_samples_split = 5, min_samples_leaf = 4, 
                                 max_features = 'sqrt', max_depth =90, random_state=42, bootstrap = False)

### Fitting Data

In [19]:
rnd_clf.fit(X_train, y_train)

C:\Users\chloe\AppData\Local\Temp\ipykernel_19964\1831517094.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rnd_clf.fit(X_train, y_train)


RandomForestClassifier(bootstrap=False, max_depth=90, min_samples_leaf=4,
                       min_samples_split=5, n_estimators=890, random_state=42)

### Predicting on train set

In [20]:
model = 'Random Forest TRAIN Tuned'
y_predict = rnd_clf.predict(X_train)
cm = confusion_matrix(y_train, y_predict)
accuracy = accuracy_score(y_train, y_predict)
precision = precision_score(y_train, y_predict, average = "weighted")
recall = recall_score(y_train, y_predict, average = "weighted")
f1 = f1_score(y_train, y_predict, average = "weighted")

In [21]:
int_metric = pd.DataFrame([[model, accuracy, precision, recall, f1, cm]], 
                          columns = ['Model', 'Accuracy', 'Precision', 'Recall', 'F1', 'Confusion Matrix'])

metrics_df = metrics_df.append(int_metric)

In [22]:
print("Confusion_Matrix:\n{}"
      .format(cm))
print("Accuracy_Score: {}\n"
      .format(accuracy))
print("Precision_Score: {}\n"
      .format(precision))
print("Recall Score: {}\n"
      .format(recall))
print("F1 Score: {}\n"
      .format(f1))

Confusion_Matrix:
[[1383    0    0]
 [   6 1375    2]
 [   0    0 1383]]
Accuracy_Score: 0.9980718245360328

Precision_Score: 0.9980787674170181

Recall Score: 0.9980718245360328

F1 Score: 0.9980707672786837



### Predicting on test set

In [23]:
model = 'Random Forest TEST Tuned'
y_predict = rnd_clf.predict(X_test)
cm = confusion_matrix(y_test, y_predict)
accuracy = accuracy_score(y_test, y_predict)
precision = precision_score(y_test, y_predict, average = "weighted")
recall = recall_score(y_test, y_predict, average = "weighted")
f1 = f1_score(y_test, y_predict, average = "weighted")

C:\Python39\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [24]:
int_metric = pd.DataFrame([[model, accuracy, precision, recall, f1, cm]], 
                          columns = ['Model', 'Accuracy', 'Precision', 'Recall', 'F1', 'Confusion Matrix'])

metrics_df = metrics_df.append(int_metric)

In [25]:
print("Confusion_Matrix:\n{}"
      .format(cm))
print("Accuracy_Score: {}\n"
      .format(accuracy))
print("Precision_Score: {}\n"
      .format(precision))
print("Recall Score: {}\n"
      .format(recall))
print("F1 Score: {}\n"
      .format(f1))

Confusion_Matrix:
[[  0  21   0]
 [  0 137   0]
 [  0  10   0]]
Accuracy_Score: 0.8154761904761905

Precision_Score: 0.6650014172335601

Recall Score: 0.8154761904761905

F1 Score: 0.7325917252146761



In [26]:
metrics_df

,Model,Accuracy,Precision,Recall,F1,Confusion Matrix
0,Random Forest TRAIN,0.955652,0.955659,0.955652,0.955349,"[[1316, 67, 0], [49, 1266, 68], [0, 0, 1383]]"
0,Random Forest TEST,0.815476,0.751709,0.815476,0.777149,"[[6, 15, 0], [5, 131, 1], [1, 9, 0]]"
0,Random Forest TRAIN Tuned,0.998072,0.998079,0.998072,0.998071,"[[1383, 0, 0], [6, 1375, 2], [0, 0, 1383]]"
0,Random Forest TEST Tuned,0.815476,0.665001,0.815476,0.732592,"[[0, 21, 0], [0, 137, 0], [0, 10, 0]]"


# Random Forest Business Metrics

## Import Data 

In [27]:
X_train = pd.read_excel("../data/model_inputs/X_train.xlsx")
y_train = pd.read_excel("../data/model_inputs/y_train.xlsx")

X_test = pd.read_excel("../data/model_inputs/X_test.xlsx")
y_test = pd.read_excel("../data/model_inputs/y_test.xlsx")

In [28]:
X_train = X_train.rename(columns ={'Unnamed: 0': 'Date'})
X_train = X_train.set_index('Date')

y_train = y_train.rename(columns ={'Unnamed: 0': 'Date'})
y_train = y_train.set_index('Date')

X_test = X_test.rename(columns ={'Unnamed: 0': 'Date'})
X_test = X_test.set_index('Date')

y_test = y_test.rename(columns ={'Unnamed: 0': 'Date'})
y_test = y_test.set_index('Date')

In [29]:
whole_df_x = pd.concat([X_train, X_test])
whole_df_x = whole_df_x.reset_index()
whole_df_x['year'] = pd.DatetimeIndex(whole_df_x['Date']).year
tuned_model = rnd_clf

In [30]:
business_metric_results = pd.DataFrame(columns=['exp_strategy_annual_return', 'exp_benchmark_annual_return', 'strategy_over_benchmark'])

for year in [2016, 2017, 2018, 2019, 2020, 2021, 2022]:
    year_data = whole_df_x[whole_df_x['year'] == year]
    year_data = year_data.set_index('Date')
    year_data = year_data.drop(['year'], axis = 1)
    predict_x = tuned_model.predict(np.array(year_data)) 
    predictions = predict_x

    df_pred = pd.DataFrame({'prediction':predictions}, index=year_data.index)
    df_pred = df_pred.replace({2:1, 1:-1}) # convert classes to buy hold sell
    dates = df_pred.index

    if year == 2022:
        end_date = "2022-09-02"
    else:
        end_date = str(year+1) + "-01-01"
    df_prices = yf.download("^GSPC", start=dates[0], end=end_date)[['Adj Close']]

    # create positions column
    positions = []
    prev = 0
    for i in range(len(df_pred)):
        positions.append(df_pred.iloc[i]['prediction'])
        # if df_pred.iloc[i]['prediction'] == 0:
        #     positions.append(prev)
        # else:
        #     prev = df_pred.iloc[i]['prediction']
        #     positions.append(prev)

    df_business = pd.DataFrame()
    df_business['stock_daily_log_return'] = np.log(df_prices /df_prices.shift(1))['Adj Close']
    df_business['prediction'] = df_pred['prediction']
    df_business['position'] = positions
    df_business['benchmark'] = 1 # long and hold strategy
    df_business["strategy_Returns"] = df_business["stock_daily_log_return"] * df_business["position"].shift(1)
    df_business["benchmark_Returns"] = df_business["stock_daily_log_return"] * df_business["benchmark"].shift(1)

    # Annual Mean Returns or Expected returns
    expected_strategy_annual_return = np.exp(df_business['strategy_Returns'].mean() * 252) - 1 
    expected_benchmark_annual_return = np.exp(df_business['benchmark_Returns'].mean() * 252) - 1 
    strategy_over_benchmark = expected_strategy_annual_return-expected_benchmark_annual_return
    business_metric_results.loc[year] = [expected_strategy_annual_return, expected_benchmark_annual_return, strategy_over_benchmark]
    # print(f'Expected Annual Returns: Strategy: {round(expected_strategy_annual_return*100, 2)}%  |  Stock: {round(expected_benchmark_annual_return*100, 2)}%')

business_metric_results

C:\Python39\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


C:\Python39\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Python39\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


C:\Python39\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Python39\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


C:\Python39\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Python39\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


[*********************100%***********************]  1 of 1 completed


,exp_strategy_annual_return,exp_benchmark_annual_return,strategy_over_benchmark
2016,0.000000,0.107616,-0.107616
2017,0.000000,0.185753,-0.185753
2018,-0.002359,-0.070634,0.068275
2019,-0.030239,0.288443,-0.318682
2020,0.159940,0.152929,0.007011
2021,0.001926,0.289230,-0.287304
2022,0.000000,-0.249185,0.249185
